In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
print ("testing")

testing


In [118]:
# Define data transformations with augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    #transforms.Resize((224, 224)),  # Resize to match ResNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [120]:
# Load the dataset from the root folder
# The root folder should contain subfolders "stable" and "unstable"
train = datasets.ImageFolder(root='10B-3D-Uni/train', transform=transform)
val = datasets.ImageFolder(root='10B-3D-Uni/val', transform=transform)
test = datasets.ImageFolder(root='10B-3D-Uni/test', transform=transform)

# Create data loaders for training and testing
train_loader = DataLoader(train, batch_size=16, shuffle=True)
val_loader = DataLoader(val, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

print(f"Training set size: {len(train)} images")
print(f"Validation set size: {len(val)} images")
print(f"Testing set size: {len(test)} images")

Training set size: 1200 images
Validation set size: 400 images
Testing set size: 400 images


In [122]:
# Load the pre-trained ResNet18 model
resnet18 = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Modify the final fully connected layer for binary classification
resnet18.fc = nn.Linear(resnet18.fc.in_features, 2)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
resnet18 = resnet18.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

Using device: cuda


In [124]:
def evaluate_model(model, data_loader):
    """Evaluate model on the provided data loader"""
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    avg_loss = running_loss / len(data_loader)
    return accuracy, avg_loss

In [126]:
# Training loop with validation

import torch.optim.lr_scheduler as lr_scheduler

num_epochs = 30
best_val_loss = float('inf')
patience = 8  # Number of epochs to wait for improvement
patience_counter = 0  # Counter for early stopping
temp = None
print("Starting training...")

# Define scheduler to reduce LR on plateauing validation loss
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

for epoch in range(num_epochs):
    # Training phase
    resnet18.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = correct / total * 100 #check underfitting
    train_loss = running_loss / len(train_loader)
    
    # Validation phase
    val_accuracy, val_loss = evaluate_model(resnet18, val_loader) #check overfitting
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # Reset the patience counter
        temp = resnet18.state_dict()
    else:
        patience_counter += 1  # Increment the patience counter
        print(f"patience counter {patience_counter}")
    # Step the scheduler with validation loss
    scheduler.step(val_loss)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}]')
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
    print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')
    print('-' * 60)

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch + 1}. Best validation loss: {best_val_loss:.4f}")
        break

torch.save(temp, '10B-3D-Uni_best_model.pth')
print("Training completed.")

Starting training...
Epoch [1/30]
Training Loss: 0.7781, Training Accuracy: 57.08%
Validation Loss: 0.7091, Validation Accuracy: 60.50%
Current LR: 0.001000
------------------------------------------------------------
Epoch [2/30]
Training Loss: 0.6524, Training Accuracy: 63.00%
Validation Loss: 0.6317, Validation Accuracy: 62.75%
Current LR: 0.001000
------------------------------------------------------------
patience counter 1
Epoch [3/30]
Training Loss: 0.6159, Training Accuracy: 66.08%
Validation Loss: 0.6518, Validation Accuracy: 63.00%
Current LR: 0.001000
------------------------------------------------------------
Epoch [4/30]
Training Loss: 0.5907, Training Accuracy: 68.08%
Validation Loss: 0.6289, Validation Accuracy: 65.50%
Current LR: 0.001000
------------------------------------------------------------
patience counter 1
Epoch [5/30]
Training Loss: 0.5680, Training Accuracy: 70.58%
Validation Loss: 0.7260, Validation Accuracy: 63.00%
Current LR: 0.001000
-----------------

In [133]:
# Final evaluation on test set
from torchvision.models import resnet18, ResNet18_Weights
# Initialize the ResNet18 model (without pre-trained weights)
model = resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

# Load fine-tuned weights
model.load_state_dict(torch.load('10B-3D-Uni_best_model.pth', weights_only=True))

# Move the model to GPU
model = model.to(device)

test_accuracy, test_loss = evaluate_model(model, test_loader)
print(f'\nFinal Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Save final model
#torch.save(model.state_dict(), 'ResNet18_final_model.pth')
#print("Final model saved!")


Final Test Loss: 0.3291, Test Accuracy: 89.25%
